<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Initialization" data-toc-modified-id="Initialization-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initialization</a></span></li><li><span><a href="#Data" data-toc-modified-id="Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data</a></span><ul class="toc-item"><li><span><a href="#Raw-Data" data-toc-modified-id="Raw-Data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Raw Data</a></span><ul class="toc-item"><li><span><a href="#Create-Regions" data-toc-modified-id="Create-Regions-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Create Regions</a></span></li><li><span><a href="#Create-(Domestic)-Variables" data-toc-modified-id="Create-(Domestic)-Variables-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Create (Domestic) Variables</a></span></li></ul></li><li><span><a href="#Construct-Trade-Weight-Matrix" data-toc-modified-id="Construct-Trade-Weight-Matrix-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Construct Trade Weight Matrix</a></span><ul class="toc-item"><li><span><a href="#Create-Country-Specific-(Starred*)-Variables" data-toc-modified-id="Create-Country-Specific-(Starred*)-Variables-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Create Country Specific (Starred*) Variables</a></span></li><li><span><a href="#Add-Regions" data-toc-modified-id="Add-Regions-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Add Regions</a></span></li></ul></li></ul></li><li><span><a href="#Analysis" data-toc-modified-id="Analysis-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Analysis</a></span><ul class="toc-item"><li><span><a href="#Error-Correction" data-toc-modified-id="Error-Correction-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Error Correction</a></span></li></ul></li><li><span><a href="#Testing" data-toc-modified-id="Testing-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Testing</a></span></li></ul></div>

# Initialization

In [1]:
import pandas as pd, numpy as np, dask.dataframe as ddf, seaborn as sns, pickle, pyflux as pf
import quandl
import sys, os, socket, string
from dask import delayed
from itertools import permutations, product
from functools import reduce
import pandas_datareader as pdr
import xlrd

scriptDir = os.path.dirname(os.path.realpath(sys.argv[0]))   # get script directory
try: 
    genDir = "C:/Users/benmo/OneDrive/GitHub/DataPlayground" #get general functions path
    os.chdir(genDir)
except: 
    genDir = "D:/benmo/OneDrive/GitHub/DataPlayground"
    os.chdir(genDir)

os.chdir(genDir)

from General import * # import custom 'general' functions
from DataEther import * # import custom 'data' functions

os.chdir(scriptDir)

In [2]:
from sympy.interactive.printing import init_printing
init_printing(use_unicode=False, wrap_line=True, no_global=True)
from sympy import Symbol, symbols, MatrixSymbol, Matrix

In [3]:
%load_ext rpy2.ipython

In [4]:
fpath = MyComp.financePath  #path to financial econ folder

cntryBook = xlrd.open_workbook(fpath + "/data/gvar/Country Data (1979Q1-2011Q2).xlsx")
sheets = cntryBook.sheet_names()

data = pd.DataFrame([])
idx = pd.IndexSlice

# Data

## Raw Data 

In [5]:
for sheet in sheets:
    temp = pd.read_excel(fpath + "/data/gvar/Country Data (1979Q1-2011Q2).xlsx", sheet_name=sheet)
    temp['Country'] = sheet
    temp.set_index([temp['Country'], temp['date']], inplace=True)
    data = data.append(temp)
    
data = data.sort_index()

tradeMatrix = pd.read_excel(fpath + "/data/gvar/Trade Matrices (1980-2010).xls")
cntryCodes = pd.read_excel(fpath + "/data/gvar/Country Codes.xls", index_col='Country')

### Create Regions

In [6]:
wEurope = ['Spain', 'Belgium', 'Netherlands', 'Switzerland']
mEast = ['Saudi Arabia', 'Turkey']
seAsia = ['Korea', 'Thailand', 'Indonesia', 'Malaysia', 'Philippines', 'Singapore']
latAmerica = ['Argentina', 'Brazil', 'Chile', 'Peru', 'Mexico']

regionDict= {}
regionDict['Western Europe'] = wEurope
regionDict['Middle East'] = mEast
regionDict['Southeast Asia'] = seAsia
regionDict['Latin America'] = latAmerica


countriesAll = ['USA', 'UK', 'Germany', 'France', 'Italy', 'China', 'Japan']
countriesAll.extend(wEurope)
countriesAll.extend(mEast)
countriesAll.extend(seAsia)
countriesAll.extend(latAmerica)


regions = ['USA', 'UK', 'Germany', 'France', 'Italy', 'China', 'Japan']
regions.extend(['Western Europe','Middle East', 'Southeast Asia', 'Latin America'])

### Create (Domestic) Variables 

In [7]:
y, q, e, p, m, rho = [{} for i in range(6)]

for region in ['USA', 'UK', 'Germany', 'France', 'Italy', 'China', 'Japan']:
    try:
        y[region] = data.loc[region][['RGDP','CPI']].apply(lambda x: np.log(x['RGDP']*x['CPI']/100), axis=1)
    except:
        pass
    try:
        p[region] = data.loc[region]['CPI'].apply(lambda x: np.log(x/100))
    except:
        pass
    try:
        q[region] = data.loc[region][['nomEQ','CPI']].apply(lambda x: np.log(x['nomEQ']*x['CPI']/100), axis=1)
    except:
        pass
    #try:
        #i_short[region] = data.loc[region][['RGDP','CPI']].apply(lambda x: np.log(x['RGDP']*x['CPI']/100), axis=1)
    #except:
        #pass
    try:
        e[region] = data.loc[region]['FXdol'].apply(lambda x: np.log(x))
    except:
        pass
    try:
        rho[region] = data.loc[region]['Rshort'].apply(lambda x: np.log(1 + x/100)) #R is based on 3 month yield
    except:
        pass


## Construct Trade Weight Matrix 

In [8]:
tradeMatrix['country_primary'] = tradeMatrix['country_primary'].apply(lambda x, code=cntryCodes: 
                                     code[code.values == x].index[0])

countries = tradeMatrix[tradeMatrix.year == 1980]['country_primary'].tolist()
tradeMatrix.rename(columns=lambda x, names=cntryCodes: 
                   names[names.values == int(x[1:])].index[0] if [x[1:]] in names
                   .values.astype(str).tolist() else x, 
                   inplace=True)



W = tradeMatrix.set_index([tradeMatrix.country_primary, 
                           pd.to_datetime(tradeMatrix.astype(str).year)])

W = W.sort_index()

### Create Country Specific (Starred*) Variables 

In [9]:
W1990 = W[['USA', 'UK', 'Germany', 'France', 'Italy', 
           'China', 'Japan']].loc[idx[[
    'USA', 'UK', 'Germany', 'France', 'Italy', 'China', 'Japan'],'1990'],:]

w = {}

for region in ['USA', 'UK', 'Germany', 'France', 'Italy', 'China', 'Japan']:
    w[region] = W1990[region].apply(lambda x, xvec=W1990[region]: x/xvec.sum())


ys, qs, es, ps, ms, rhos = [{} for i in range(6)]   

global N

N = len(y['USA'])

def get_weighted(var, w_reg):
    return sum([var[region] * w_reg[region].values[0] for region in w_reg.dropna().index.get_level_values(0)])



def get_starred(var, temp):
    star = {}
    for region in var.keys():
        star[region] = []
        for i in range(N):
            tempi = temp[list(filter(lambda x: x != region, temp.iloc[i].index))].iloc[i]
            star[region].append(get_weighted(tempi, w[region]))

    return pd.DataFrame(star, index=pd.DataFrame(var).index)    

        
        

ys, qs, es, ps, rhos = [get_starred(x, pd.DataFrame(x)) for x in [y,q,e,p,rho]]
y, q, e, p, rho = [pd.DataFrame(x) for x in [y,q,e,p,rho]]





### Add Regions 

In [ ]:

W = W.append(pd.DataFrame(index=list(product(['Western Europe','Middle East',
                                         'Southeast Asia', 'Latin America'],
                                             list(map(str,range(1980,2011)))))))

for key, val in regionDict.items():
    W[key] = W[list(filter(lambda x: x in val, 
                                        W.columns.tolist()))].apply(sum, axis=1)
    for country in countries:
        W.loc[idx[key,:], country] = W[key][country].values
    
for key, val in regionDict.items():    
    for region in list(filter(lambda x: x!= key, regions[-4:])):
        W.loc[idx[key,:], region] = W.loc[idx[key,:],
                                          regionDict[region]].apply(sum, axis=1)    

    
W = W.sort_index()


W = W[regions]                            # Only include countries/regions used in paper

W = W.apply(lambda x: x/x.sum(), axis=1)  # Convert dollar value trade data matrix to weights matrix

# Analysis

## Error Correction 

In [10]:
%R library(MTS)
%R library(urca)
#%R library('GVAR', lib.loc='C:/Users/benmo/Documents/R/win-library/3.4')

array(['urca', 'MTS', 'tools', 'stats', 'graphics', 'grDevices', 'utils',
       'datasets', 'methods', 'base'],
      dtype='<U9')

In [39]:
%R -i y y <- data.frame(y)
%R -i e e <- data.frame(e)
%R -i p p <- data.frame(p)
%R -i rho rho <- data.frame(rho)

%R china <- data.frame(y['China'], e['China'], p['China'], rho['China'])
%R names(china) <- c('y', 'e', 'p', 'rho')
x = %R capture.output(ECMvar(china, 2, c(1,1,1,1)))
for line in x: print(line)



Order p:  2  Co-integrating rank:  1 
Number of parameters:  23 
initial estimates:  -0.0001612138 -0.002522158 -0.0004090163 -0.0001167729 1 1 1 0.002175153 -0.002252049 -0.003076825 -0.007182022 0.03402985 -0.03523286 -0.04813632 -0.1123613 0.005518593 -0.005713685 -0.007806227 -0.01822154 0.001575542 -0.00163124 -0.002228655 -0.005202197 
Par. Lower-bounds:  -0.001281439 -0.0037349 -0.0009637744 -0.0002523696 0.9069739 0.9069739 0.9069739 -0.1712662 -0.09231841 -0.3633811 -0.8207534 -0.1537356 -0.1327376 -0.4381973 -0.9931236 -0.08037306 -0.05031636 -0.1862362 -0.4211184 -0.01941851 -0.01253324 -0.04584137 -0.1036802 
Par. Upper-bounds:  0.0009590116 -0.001309416 0.0001457419 1.882374e-05 1.093026 1.093026 1.093026 0.1756165 0.08781432 0.3572275 0.8063893 0.2217953 0.06227187 0.3419247 0.768401 0.09141025 0.03888899 0.1706237 0.3846753 0.0225696 0.009270762 0.04138406 0.09327582 
Final   Estimates:  0.0009590116 -0.001309416 -0.0003501017 -0.0001754271 1.093026 0.9069739 1.093026 -0

# Testing

In [ ]:
W1 = pd.DataFrame(index=regions, columns=regions) 

for region in regions:
    W1[region] = W.loc[idx[region, '1996':'1998'], :].apply(np.mean, axis=0)
    
W1

In [ ]:
W1['USA'].sum()
W1.loc['USA'].sum()

In [ ]:
data.loc['USA'].head()

In [19]:
W1990 = W[['USA', 'UK', 'Germany', 'France', 'Italy', 
           'China', 'Japan']].loc[idx[[
    'USA', 'UK', 'Germany', 'France', 'Italy', 'China', 'Japan'],'1990'],:]
W1990

,,USA,UK,Germany,France,Italy,China,Japan
country_primary,year,,,,,,,
China,1990-01-01,5952.470,1023.5275,2521.03,1163.614,979.737,NaN,8433.18
France,1990-01-01,15848.200,18190.2000,40376.95,NaN,25360.100,1809.2000,6696.55
Germany,1990-01-01,25973.800,28540.3500,NaN,46425.800,34781.650,3681.1750,15641.05
Italy,1990-01-01,11123.095,10762.6600,35661.80,26844.600,NaN,1401.2625,4107.34
Japan,1990-01-01,71981.300,8037.7650,14784.25,6878.730,4219.480,9100.7150,NaN
UK,1990-01-01,24158.050,NaN,29576.45,20110.800,10948.950,929.4535,8386.68
USA,1990-01-01,NaN,22207.8000,23916.20,13622.950,10691.300,10551.5000,70827.15


In [47]:
bob=W1990['USA'].apply(lambda x, xvec=W1990['USA']: x/xvec.sum())
bob.index.get_level_values(0).tolist()

['China', 'France', 'Germany', 'Italy', 'Japan', 'UK', 'USA']

In [66]:
pd.DataFrame(e)

,China,France,Germany,Italy,Japan,UK,USA
date,,,,,,,
1979Q1,0.452330,-0.413018,-0.035454,-0.821217,5.290197,-0.710212,0.0
1979Q2,0.457078,-0.388494,-0.014047,-0.811855,5.367343,-0.741518,0.0
1979Q3,0.430000,-0.421087,-0.056393,-0.848312,5.373039,-0.811744,0.0
1979Q4,0.415753,-0.443273,-0.084389,-0.843459,5.459491,-0.778428,0.0
1980Q1,0.412985,-0.441335,-0.080224,-0.838397,5.479900,-0.821534,0.0
1980Q2,0.403568,-0.426273,-0.059503,-0.806606,5.434314,-0.834913,0.0
1980Q3,0.378949,-0.448041,-0.079002,-0.816062,5.378611,-0.876854,0.0
1980Q4,0.411730,-0.378169,-0.005407,-0.744446,5.334790,-0.878520,0.0
1981Q1,0.464878,-0.283251,0.082427,-0.644445,5.310393,-0.845473,0.0


In [107]:
w['China'].dropna()[idx['France']].values[0]

In [113]:
w['China'].dropna().index.get_level_values(0)

get_eTilda = lambda x, y: y - sum([x.dropna()[idx[region]].values[0] for region in x.dropna().index.get_level_values(0)])

In [130]:
bob=pd.DataFrame(e)

bob.iloc[0]['China']

In [40]:
%R -i y y <- data.frame(y)
%R -i e e <- data.frame(e)
%R -i p p <- data.frame(p)
%R -i rho rho <- data.frame(rho)

%R china <- data.frame(y['China'], e['China'], p['China'], rho['China'])
%R names(china) <- c('y', 'e', 'p', 'rho')
x = %R capture.output(ECMvar(china, 2, c(1,1,1,1)))
for line in x: print(line)

Order p:  2  Co-integrating rank:  1 
Number of parameters:  23 
initial estimates:  -0.0001612138 -0.002522158 -0.0004090163 -0.0001167729 1 1 1 0.002175153 -0.002252049 -0.003076825 -0.007182022 0.03402985 -0.03523286 -0.04813632 -0.1123613 0.005518593 -0.005713685 -0.007806227 -0.01822154 0.001575542 -0.00163124 -0.002228655 -0.005202197 
Par. Lower-bounds:  -0.001281439 -0.0037349 -0.0009637744 -0.0002523696 0.9069739 0.9069739 0.9069739 -0.1712662 -0.09231841 -0.3633811 -0.8207534 -0.1537356 -0.1327376 -0.4381973 -0.9931236 -0.08037306 -0.05031636 -0.1862362 -0.4211184 -0.01941851 -0.01253324 -0.04584137 -0.1036802 
Par. Upper-bounds:  0.0009590116 -0.001309416 0.0001457419 1.882374e-05 1.093026 1.093026 1.093026 0.1756165 0.08781432 0.3572275 0.8063893 0.2217953 0.06227187 0.3419247 0.768401 0.09141025 0.03888899 0.1706237 0.3846753 0.0225696 0.009270762 0.04138406 0.09327582 
Final   Estimates:  0.0009590116 -0.001309416 -0.0003501017 -0.0001754271 1.093026 0.9069739 1.093026 -0

In [41]:
%R install.packages('urca')

In [31]:
%R -i y y <- data.frame(y)
%R -i e e <- data.frame(e)
%R -i p p <- data.frame(p)
%R -i rho rho <- data.frame(rho)

%R china <- data.frame(y['China'], e['China'], p['China'], rho['China'])
%R names(china) <- c('y', 'e', 'p', 'rho')
#%R chinaR <- read.csv('c:/users/benmo/desktop/test1.csv', header=TRUE)
%R -n bob <- ca.jo(china, type = c('eigen'), ecdet = c('none'), K = 2,spec=c('longrun'), season = NULL, dumvar = NULL)
x = %R capture.output(summary(bob))
for line in x:
    print(line)


###################### 
# Johansen-Procedure # 
###################### 

Test type: maximal eigenvalue statistic (lambda max) , with linear trend 

Eigenvalues (lambda):
[1] 0.2293384385 0.1096654761 0.0907079693 0.0004714185

Values of teststatistic and critical values of test:

          test 10pct  5pct  1pct
r <= 3 |  0.06  6.50  8.18 11.65
r <= 2 | 12.17 12.91 14.90 19.19
r <= 1 | 14.87 18.90 21.07 25.75
r = 0  | 33.34 24.78 27.14 32.14

Eigenvectors, normalised to first column:
(These are the cointegration relations)

              y.l2      e.l2        p.l2     rho.l2
y.l2      1.000000  1.000000    1.000000  1.0000000
e.l2      3.840216  1.611148  -51.916588 -0.9032136
p.l2     -7.296459 -4.130745   -2.132475 -1.2816761
rho.l2 -103.799465  4.763804 -177.859468  9.3241144

Weights W:
(This is the loading matrix)

               y.l2         e.l2          p.l2        rho.l2
y.d    0.0011268953 1.357076e-02 -2.108745e-05  2.614037e-04
e.d   -0.0055963757 1.861738e-02  2.962646e-0

In [25]:
%%R
a= list()
for(i in 1:8){
    a[[i]] = 3
}

In [29]:
x = %R a[[1]]
x[0]

In [28]:
bob = pd.DataFrame([])
for x in [y['China'],e['China'],p['China'],rho['China']]:
    bob = pd.concat([bob, pd.DataFrame(x)], axis=1)
bob.columns=['y', 'e', 'p', 'rho']

In [31]:
bob.to_csv("c:/users/benmo/desktop/test1.csv", index=False)